In [1]:
import numpy as np
import pandas as pd
import os
import re

import time
import platform

import bs4
import requests

import urllib
from urllib.request import urlopen

import selenium
from selenium import webdriver as wd
from selenium.webdriver import ActionChains

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait

from IPython.display import clear_output

# 한겨레 뉴스 링크 크롤링 함수 생성

In [2]:
# 요청 함수
def getSource(site) :
    # 헤더 정보
    header_info = {
        'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWeb Kit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.146 Safari/537.36'
    }
    
    # 요청한다.
    response = requests.get(site, headers=header_info)
    
    # bs4 객체 생성
    soup = bs4.BeautifulSoup(response.text, 'lxml')
    
    return soup

In [3]:
# 한 페이지에 있는 다음 뉴스 링크 수집 함수
def getNewsLink(soup):

    link_list = []

    # li 태그 가져오기
    a1 = soup.select('#clusterResultUL > li')
    # print(len(a1))
    
    for a2 in a1:
        
        # div 태그 가져오기
        a3 = a2.select_one('div.wrap_cont > div > div > a')
        # print(a3)
        
        # 기사링크 
        data1 = a3.attrs['href']
        # print(data1)
        
        # 기사제목
        data2 = a3.text.strip()
        # print(data2)
    
        # span 태그 가져오기
        a4 = a2.select_one('div.wrap_cont > div > span.f_nb.date')
        a5 = a4.text.strip().split('|')
        
        # 날짜
        data3 = a5[0]
        
        # 언론사
        data4 = a5[1]
        
        # print(data1, data2, data3, data4)
        
        # 기사 링크 리스트에 저장
        link_list.append(data1)
    
    # 데이터프레임 생성
    df1 = pd.DataFrame(link_list)
    display(df1)
    
    FILENAME = 'hani_link.csv'
    if os.path.exists(FILENAME) == False:
        # 파일이 없을 경우
        df1.to_csv(FILENAME, encoding='utf-8-sig', index=False)
    else:
        # mode='a' : 기존 것 뒤에다 붙여줌
        df1.to_csv(FILENAME, encoding='utf-8-sig', index=False, header=False, mode='a')
    
        
    return link_list

In [4]:
# 다음 페이지 존재 여부 확인하는 함수
def getNextPage(site) :
    
    # url에서 p= 값 들고오기
    p = site.split('&')[-1].split('=')[-1]
    
    # p값에 1 더해서 다음 페이지 url 만들기
    nextPage = site[:-len(p)] + str(int(p)+1)

    # 현재 페이지와 다음 페이지 soup 가져오기
    soup1 = getSource(site)
    soup2 = getSource(nextPage)
    # print(soup1)
    # print(soup2)

    # 현재 페이지와 다음 페이지 첫번째 a 태그에서 링크 가져오기
    a1 = soup1.select('#clusterResultUL > li > div.wrap_cont > div > div > a')[0].attrs['href']
    a2 = soup2.select('#clusterResultUL > li > div.wrap_cont > div > div > a')[0].attrs['href']
    print(a1)
    print(a2)
    
    # 두 링크가 같지 않으면 다음 페이지가 있다고 간주, 다음 페이지 return 
    if a1 != a2 :
        return True
    # 같으면 다음 페이지 없다고 간주, False return 
    else :
        return False

# 한겨레 url을 담은 csv 파일 저장

In [5]:
KEYWORD = urllib.parse.quote('보궐선거')
COLOPHON = '한겨레'

dayStart = '20210301000001'
dayEnd = '20210406120000'

page = 1
URL = 'https://search.daum.net/search?w=news&enc=utf8&cluster=y&cluster_page=1&'
cp_dict = {'조선일보' : '16d4PV266g2j-N3GYq',
           '중앙일보' : '16Elf9uX5H6T5xXvQV',
           '동아일보' : '16bOiOx4gG2S18EPLj',
           'JTBC'     : '16yZfDfR_rGcw5F-P0',
           '경향신문' : '16akMkKFDu6n8GTzZr',
           '한겨레' : '16nzyJHdH5ORpabfqG'}
cpName = urllib.parse.quote(COLOPHON)
cp = cp_dict[COLOPHON]

while True :
    time.sleep(1)
            
    clear_output(wait=True)
        
    site = f'{URL}q={KEYWORD}&cpname={cpName}&cp={cp}&period=6m&sd={dayStart}&ed={dayEnd}&DA=PGD&p={page}'

    print(f' 다음 뉴스 - {COLOPHON} : {page} 페이지 수집 중' )
    
    soup = getSource(site)
    getNewsLink(soup) 
    chk = getNextPage(site)

    if chk != False:
        page = page + 1
    else: 
        print('수집완료')
        break

 다음 뉴스 - 한겨레 : 45 페이지 수집 중


,0
0,http://v.media.daum.net/v/20210316200604054?f=o
1,http://v.media.daum.net/v/20210315205607403?f=o
2,http://v.media.daum.net/v/20210314101604196?f=o
3,http://v.media.daum.net/v/20210307135609306?f=o
4,http://v.media.daum.net/v/20210303165614030?f=o
5,http://v.media.daum.net/v/20210304162601513?f=o


http://v.media.daum.net/v/20210316200604054?f=o
http://v.media.daum.net/v/20210316200604054?f=o
수집완료


# 한겨레 댓글 크롤링 함수 생성

In [2]:
# 댓글 iframe 가져오는 함수
def getCommentIframe():
    # 댓글 iframe이 없을 경우 처리가가 복잡해지니 iframe이 나올 때 까지 반복한다.
    chk = True

    while chk :
        time.sleep(1)

        # 원하는 기사를 요청한다.

        # 가장 아래로 스크롤 한다.
        # 스크롤을 해야 댓글이 나옴
        driver.execute_script("window.scrollTo(0, 50000)") 

        # iframe들을 모두 가져온다.
        iframe_list = driver.find_elements_by_tag_name('iframe')

        # iframe의 수 만큼 반복한다.
        for a1 in iframe_list :
            # iframe의 title 속성값을 가져온다.
            title1 = a1.get_attribute('title')

            # title 속성값이 livere-comment 라면 src 속성을 추출한다.
            if title1 == 'livere-comment' :
                src1 = a1.get_attribute('src')
                # print(src1)
                return src1
                # 댓글 iframe을 발견하였으므로 while이 종료될 수 있게 한다.
                chk = False

In [3]:
# 더보기 계속 클릭하기
def clickMore(driver):
            
   # 댓글 더보기 클릭
    while True :
        try :
            time.sleep(1)
            driver.find_element_by_class_name('more-btn').click()
            # print('클릭')

        except :
            # print('클릭 끝')
            break

In [4]:
def getComment(driver, data1, data2) :
    
    # 데이터를 담을 딕셔너리
    comment_dict = {
        '제목' : [],
        '날짜' : [],
        '작성일' : [],
        '댓글' : []
    }
    
    a1 = driver.find_elements_by_tag_name('#list > div.reply-wrapper')
    # 댓글이 없으면
    if not a1:
        return False
        
    else:
        
        for a2 in a1:
            a3 = a2.find_elements_by_tag_name('div.reply-top > div > ul')[0]
            
            # 사용자 id 
            a4 = a3.find_elements_by_tag_name('li.writer-name')[0].text
            if a4 == '삭제된 댓글입니다.':
                print(a4)
                continue

            # 작성일
            data3 = a3.find_elements_by_tag_name('li.reply-history > div > span')[0].text
            print(data3)

            # 댓글 & 답글 태그
            a5 = a2.find_elements_by_tag_name('div.reply-bottom')[0]
            
            # 답글 유무 확인
            reply = a5.find_elements_by_tag_name('div.reply-content-wrapper > div.reply-btn-group > div.left > button')[0].text
            reply_css_selector = 'div.child-reply'
            # print('응답 시작 : ' + reply)
            
            if reply == '댓글보기' or reply == '':
                continue
                        

            # 댓글 내용
            data4 = a5.find_elements_by_tag_name('div.reply-content-wrapper > div.reply-content > p')[0].text
            
            comment_dict['제목'].append(data1)
            comment_dict['날짜'].append(data2)
            comment_dict['작성일'].append(data3)
            comment_dict['댓글'].append(data4)
                
        df = pd.DataFrame(comment_dict)
        display(df)
        
        FILENAME = 'hani_comment.csv'
        if os.path.exists(FILENAME) == False:
            # 파일이 없을 경우
            df.to_csv(FILENAME, encoding='utf-8-sig', index=False)
        else:
            # mode='a' : 기존 것 뒤에다 붙여줌
            df.to_csv(FILENAME, encoding='utf-8-sig', index=False, header=False, mode='a')

        return 1

In [5]:
# 기사 제목 & 날짜 수집
def getTitleDate(driver):
    # 기사 제목
    
    title = driver.find_element_by_css_selector('#article_view_headline > h4 > span').text
    
    # 기사 날짜
    date = driver.find_element_by_css_selector('#article_view_headline > p.date-time > span:nth-child(1)').text
    # print(date[4:])
    
    return title, date[4:]

In [6]:
# 추출한 데이터를 csv 파일로 저장하는 함수
def saveData(driver):
    
    # 기사의 제목과 날짜를 가져온다
    data1, data2 = getTitleDate(driver)
    # 기사의 댓글 iframe을 가져온다
    src1 = getCommentIframe()

    # 댓글 페이지를 요청한다.
    driver.get(url=src1)
    # 더보기 클릭
    clickMore(driver)
    # 데이터 들고오기
    chk = getComment(driver, data1, data2)
    
    return chk

# 한겨레 댓글 크롤링 후 저장

In [ ]:
link_df = pd.read_csv('hani_link.csv')
num_link = link_df.shape[0]
# print(link_df.loc[0])

# 웹 드라이버
driver = wd.Chrome('./chromedriver.exe')
idx = 0

while True :
    
    driver.implicitly_wait(20)
       
    clear_output(wait=True)
    
    driver.get(link_df.loc[idx].values[0])

    print(f'{idx}번째 기사 댓글 수집 중')

    chk = saveData(driver)
    if chk == 1:
        print(f'{idx}번째 기사 댓글 정상 수집 완료')
    else:
        print(f'{idx}번째 기사 댓글 없음, 다음 기사로')
    
    idx = idx + 1
    if idx == num_link :
        break

         
print('수집완료')

300번째 기사 댓글 수집 중
